In [1]:
from glob import glob
import pandas as pd
import os
import re

In [2]:
data_folder = "../data/thesis quantative results"
sampling_folders = glob(data_folder+"/*")
print(sampling_folders[:5])
print(f"We have {len(sampling_folders)} samples")

sampling_num_interiors = []
sampling_num_CH = []
sampling_num_puzzles = []
sampling_total_num_points = []
sampling_ids = []

for folder in  sampling_folders:
    csv_paths = glob(os.path.join(folder,"*.csv"))
    count_puzzles = 0
    folder_puzzles_paths = []
    is_metdata_found = False

    num_interior = None
    num_CH = None

    for csv_path in csv_paths:
        csv_name = os.path.basename(csv_path)

        # Because the first kernel has always tiling, the metadata always starts with a letter 
        # There should be only one, if there many, take the latest (the variables num_interior and num_CH are overrided)
        if re.match("^[a-zA-Z]+.*",csv_name) and not csv_name.startswith("s_"):
            is_metdata_found = True
            df_metadata = pd.read_csv(csv_path)
            counts_roles = df_metadata["role"].value_counts()
            try:
                num_interior = counts_roles["interior"]
                # sampling_num_interiors.append()
            except KeyError:
                num_interior = 0
                # sampling_num_interiors.append(0)
            
            num_CH = counts_roles["frame"]
            # sampling_num_CH.append(counts_roles["frame"])
        else:
            count_puzzles+=1
    

    if not is_metdata_found:
        raise Exception(f"Cannot find the sampling metadata csv of {folder}")
    
    sampling_num_interiors.append(num_interior)
    sampling_num_CH.append(num_CH)

    sampling_num_puzzles.append(count_puzzles)
    containing_dir = os.path.basename(folder)
    sampling_ids.append(containing_dir)



df_all_samples = pd.DataFrame({
    "id": sampling_ids,
    "num_interior":sampling_num_interiors,
    "num_CH": sampling_num_CH,
    "num_puzzles":sampling_num_puzzles
})

df_all_samples["num_total_points"] = df_all_samples["num_interior"] + df_all_samples["num_CH"]

# A sampling must create a puzzle, remove all the sampling that were failed
df_all_samples.drop(df_all_samples[df_all_samples["num_puzzles"] == 0].index, inplace=True)
df_all_samples.head(50)

['../data/thesis quantative results\\2024-06-25 20-16-51', '../data/thesis quantative results\\2024-06-25 20-34-35', '../data/thesis quantative results\\2024-06-25 20-34-37', '../data/thesis quantative results\\2024-06-25 20-34-40', '../data/thesis quantative results\\2024-06-25 20-34-44']
We have 754 samples


,id,num_interior,num_CH,num_puzzles,num_total_points
0,2024-06-25 20-16-51,5,5,6439,10
1,2024-06-25 20-34-35,2,5,61,7
2,2024-06-25 20-34-37,2,5,109,7
3,2024-06-25 20-34-40,1,6,155,7
4,2024-06-25 20-34-44,3,4,59,7
5,2024-06-25 20-34-46,2,6,471,8
6,2024-06-25 20-35-07,3,5,793,8
7,2024-06-25 20-35-45,4,4,579,8
8,2024-06-25 20-36-29,2,6,371,8
9,2024-06-25 20-39-07,3,7,24255,10


In [4]:
df_all_samples.groupby(["num_CH","num_interior"],as_index=False).agg({"num_puzzles":["mean","std","count"]})

num_CH num_interior   num_puzzles                   
                                mean          std count
0       3            0      1.000000          NaN     1
1       3            1      1.000000          NaN     1
2       3            2      3.000000     0.000000    50
3       3            3     15.000000          NaN     1
4       3            4     63.823529    27.238727    51
5       4            0      3.000000          NaN     1
6       4            1      5.000000          NaN     1
7       4            2     17.041667     4.598604    48
8       4            3     77.440678    29.609103    59
9       4            4    345.285714   124.854201     7
10      4            5   1793.000000          NaN     1
11      4            6   8591.666667  5134.293070     3
12      5            0     11.000000          NaN     1
13      5            1     27.678571     6.543606    28
14      5            2     93.055556    19.982157   108
15      5            3    465.720000   157.043480    50
16      5            4   2184.142857   740.168321     7
17      5            5  10940.200000  4536.642733    10
18      6            0     45.000000          NaN     1
19      6            1    130.040000    19.370670    50
20      6            2    531.880000   135.678588    50
21      6            3   2689.294118   874.398131    51
22      6            4  15473.000000  3558.621756     6
23      7            0    197.000000     0.000000    50
24      7            1    664.000000   118.720961     4
25      7            2   3417.880000   825.990479    50
26      7            3  17445.461538  6451.182329    13
27      8            1   3721.000000          NaN     1
28      8            2  19771.560000  4826.123455    50

In [5]:
df_all_samples[(df_all_samples["num_CH"] == 7) & (df_all_samples["num_interior"] == 0)]

,id,num_interior,num_CH,num_puzzles,num_total_points
49,2024-06-26 00-29-53,0,7,197,7
73,2024-06-26 00-33-23,0,7,197,7
144,2024-06-26 00-42-00,0,7,197,7
318,2024-06-26 11-06-42 on-circle,0,7,197,7
319,2024-06-26 11-06-51 on-circle,0,7,197,7
320,2024-06-26 11-07-01 on-circle,0,7,197,7
321,2024-06-26 11-07-26 on-circle,0,7,197,7
322,2024-06-26 11-16-56 on-circle,0,7,197,7
323,2024-06-26 11-17-06 on-circle,0,7,197,7
324,2024-06-26 11-17-22 on-circle,0,7,197,7


In [7]:
df_all_samples[(df_all_samples["num_CH"] == 6) & (df_all_samples["num_interior"] == 1)]

,id,num_interior,num_CH,num_puzzles,num_total_points
3,2024-06-25 20-34-40,1,6,155,7
15,2024-06-26 00-24-49,1,6,155,7
16,2024-06-26 00-24-58,1,6,155,7
17,2024-06-26 00-25-08,1,6,155,7
18,2024-06-26 00-25-18,1,6,131,7
19,2024-06-26 00-25-30,1,6,131,7
22,2024-06-26 00-25-54,1,6,107,7
38,2024-06-26 00-28-07,1,6,107,7
42,2024-06-26 00-28-45,1,6,107,7
46,2024-06-26 00-29-21,1,6,131,7


In [17]:
df_all_samples.groupby(["num_interior","num_CH"]).size()

num_interior  num_CH
0             3           1
              4           1
              5           1
              6           1
              7          50
1             3           1
              4           1
              5           3
              6          50
              7           4
              8           1
2             4           3
              5         108
              6          23
              7           8
              8           2
3             3           1
              4          59
              5          50
              6          18
              7           5
4             3          51
              4           7
              5           7
              6           6
5             4           1
              5          10
6             4           3
dtype: int64

In [44]:
ch_5_int_2 = df_all_samples[(df_all_samples["num_CH"] == 5) & (df_all_samples["num_interior"] == 2)]
ch_5_int_2.head()

,id,num_interior,num_CH,num_puzzles,num_total_points
1,2024-06-25 20-34-35,2,5,61,7
2,2024-06-25 20-34-37,2,5,109,7
14,2024-06-26 00-24-41,2,5,77,7
21,2024-06-26 00-25-47,2,5,77,7
23,2024-06-26 00-26-06,2,5,109,7


In [49]:
print("For the first quantative experiment (table)")
print(ch_5_int_2[:50]["num_puzzles"].mean())
print(ch_5_int_2[:50]["num_puzzles"].std())

# For the second quantative experiment (table)
print("For the second quantative experiment (table)")
print(ch_5_int_2[50:100]["num_puzzles"].mean())
print(ch_5_int_2[50:100]["num_puzzles"].std())

For the first quantative experiment (table)
92.52
22.937572776951775
For the second quantative experiment (table)
92.52
17.80878022384533
